# Comparison of Various Machine Learning Models for Handwritten Character Recognition
This is our Jupiter Notebook run the code we used to produce results step-by-step.

Import necessary helper functions.

In [ ]:
import pandas as pd
from testing_models import evaluate_model, test_model_nn
import matplotlib.pyplot as plt
from preprocess import get_data
from classifiers import *
from nn import *
from joblib import load
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from skopt.space import Integer
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 1 Data
We used data from the Alpha_Num dataset on kaggle. It contains over 108000 images of handwritten characters. Each image is approximately 28x28 pixels and is in gray-scale. However, before we can train the models we preprocessed them to ensure each image had the same features:
* 28x28 pixels: any image less than 28x28 was padded
* Gray-Scale

For more information on getting data please refer to **preprocess.py**

In [ ]:
X_train, y_train = get_data("train", "ascii_file_counts.csv")
X_test, y_test = get_data("test", "ascii_file_counts.csv")

# 2 Training Traditional Machine Learning Models
In this section we will train (or load) and show a brief testing of the following models:
* XGBoost
* Random Forest
* K-Nearest Neighbors

The actual results and comparision of models will be done after this section where models are trained (or loaded)

## 2.1 Training Random Forest

In [ ]:
RF_model = RandomForestClassifier()

param_space = {
    'n_estimators': (10, 500),  # Number of trees
    'max_depth': (1, 100),  # Maximum depth
    'min_samples_split': (2, 20),  # Minimum number of samples to split
    'min_samples_leaf': (1, 20),  # Minimum number of samples to be leaf
    'max_features': ['sqrt', 'log2', None],  # Features to consider
    'criterion': ['gini', 'entropy', 'log_loss'],  # Measure for split quality
    'class_weight': ['balanced', 'balanced_subsample', None],  # Class weights for handling imbalances
}

bayes_opt = BayesSearchCV(
    estimator=RF_model,
    search_spaces=param_space,
    n_iter=30,
    cv=5,       # 5-folds
    scoring='neg_mean_squared_error',  # Objective function to minimize MSE
    n_jobs=-1
)

We offer 2 methods to get the Random Forest Model. We trained the model using the code block with the training loop. However, this takes time, so if you want you can directly load the model we provided using the second code block.

In [ ]:
bayes_opt.fit(X_train, y_train)
RF_model = bayes_opt.best_estimator_
RF_bayes_df = pd.DataFrame(bayes_opt.cv_results_)

In [ ]:
RF_data = pd.read_csv("RF_bayes_df.csv")
RF_model = RandomForestClassifier(class_weight= 'balanced', criterion= 'log_loss', max_depth= 41, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 6, n_estimators= 497, random_state =42)
RF_model.fit(X_train, y_train)

In [ ]:
f1_list = []
acc_list = []
prec_list = []
recall_list = []

In [ ]:
f1, acc, cm, prec, recall = evaluate_model(y_test, RF_model.predict(X_test))

f1_list.append(f1)
acc_list.append(acc)
prec_list.append(prec)
recall_list.append(recall)

print(f"Random Forest: \n")
print(f"F1 Score: {f1}")
print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {recall}")
print(f"Confusion Matrix: \n{cm}")

## 2.2 Training XGBoost

In [ ]:
XG_model = XGBClassifier(objective='multi:softprob',num_class=93,booster='gbtree',eval_metric= 'mlogloss')

param_space = {
    'n_estimators': Integer(50, 300),        # Number of trees
    'max_depth': Integer(3, 30),             # Depth of each tree     
}

bayes_opt = BayesSearchCV(
    estimator=XG_model,
    search_spaces=param_space,
    n_iter=30,
    cv=5,       # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Objective function: MSE
    n_jobs=-1,
)

Training code

In [ ]:
bayes_opt.fit(X_train, y_train)
XG_model = bayes_opt.best_estimator_
XG_bayes_df = pd.DataFrame(bayes_opt.cv_results_)

Loading trained model code

In [ ]:
XG_model = load('xgboost.joblib')

In [ ]:
f1, acc, cm, prec, recall = evaluate_model(y_test, XG_model.predict(X_test))

f1_list.append(f1)
acc_list.append(acc)
prec_list.append(prec)
recall_list.append(recall)

print(f"XGBoost: \n")
print(f"F1 Score: {f1}")
print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {recall}")
print(f"Confusion Matrix: \n{cm}")

## 2.3 Training KNN

In [ ]:
KNN_model = KNeighborsClassifier(weights="distance")

param_space = {
    'n_neighbors': Integer(1,200)      # Minimum samples per leaf
}

bayes_opt = BayesSearchCV(
    estimator= KNN_model,
    search_spaces=param_space,
    n_iter=20, 
    cv=5,    
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

Training code

In [ ]:
bayes_opt.fit(X_train, y_train)
KNN_model = bayes_opt.best_estimator_
KNN_bayes_df = pd.DataFrame(bayes_opt.cv_results_)

Loading trained model code

In [ ]:
KNN_model = load('knn_model.pkl')

In [ ]:
f1, acc, cm, prec, recall = evaluate_model(y_test, KNN_model.predict(X_test))

f1_list.append(f1)
acc_list.append(acc)
prec_list.append(prec)
recall_list.append(recall)

print(f"KNN: \n")
print(f"F1 Score: {f1}")
print(f"Accuracy: {acc}")
print(f"Precision: {prec}")
print(f"Recall: {recall}")
print(f"Confusion Matrix: \n{cm}")

# 3 Training Neural Network Models
In this section we will train (or load) and show a brief testing of the following models:
* Feed Forward Neural Network
* CNN (Convolutional Neural Network )
* Transformer (With CNN features)

The details of each model and the PyTorch implementation as well as the training loop details can be found in the **nn.py** file.

## Initializing Data

In [ ]:
input_size = 28 * 28
num_classes = 93
learning_rate = 0.001
num_epochs = 20
batch_size = 64

dataset = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="train")
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataset_test = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
val_dataset = AlphaNumDataset(csv_dir="ascii_file_counts.csv", data_dir="validation")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


## 3.1 Training Feed Forward Neural Network

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FF_model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(FF_model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)
best_val_loss = float('inf')
patience = 3
counter = 0

Training

In [ ]:
train_model(num_epochs, data_loader, val_loader, device, FF_model, criterion, optimizer, scheduler, patience)

Loading trained model

In [ ]:
FF_model = torch.load('FeedForward.pth')

Testing + Results

In [ ]:
f1, acc, prec, recall = test_model_nn(FF_model, data_loader_test, device, criterion)

f1_list.append(f1)
acc_list.append(acc)
prec_list.append(prec)
recall_list.append(recall)

print(f"Feed Forward Neural Network: \n")
print(f"F1 Score (Weighted Among Classes): {f1}")
print(f"Accuracy: {acc}")
print(f"Precision (Weighted Among Classes): {prec}")
print(f"Recalln (Weighted Among Classes): {recall}")

## 3.2 Training Convolutional Neural Network

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CNN_model = CNN(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CNN_model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)
best_val_loss = float('inf')
patience = 3
counter = 0


Training

In [ ]:
train_model(num_epochs, data_loader, val_loader, device, CNN_model, criterion, optimizer, scheduler, patience)

Loading trained model

In [ ]:
CNN_model = torch.load('cnn.pth')

Testing + Results

In [ ]:
f1,acc,prec,recall = test_model_nn(CNN_model,data_loader_test, device, criterion)

f1_list.append(f1)
acc_list.append(acc)
prec_list.append(prec)
recall_list.append(recall)

print(f"Feed Forward Neural Network: \n")
print(f"F1 Score (Weighted Among Classes): {f1}")
print(f"Accuracy: {acc}")
print(f"Precision (Weighted Among Classes): {prec}")
print(f"Recalln (Weighted Among Classes): {recall}")

## 3.3 Training Transformer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer_model = CNNTransformer(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(transformer_model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

Training

In [ ]:
train_model(num_epochs, data_loader, val_loader, device, transformer_model, criterion, optimizer, scheduler, patience)

Loading trained model

In [ ]:
transformer_model = torch.load('transformer.pth')

Testing + Results

In [ ]:
f1,acc,prec,recall = test_model_nn(transformer_model,data_loader_test, device, criterion)

f1_list.append(f1)
acc_list.append(acc)
prec_list.append(prec)
recall_list.append(recall)

print(f"Feed Forward Neural Network: \n")
print(f"F1 Score (Weighted Among Classes): {f1}")
print(f"Accuracy: {acc}")
print(f"Precision (Weighted Among Classes): {prec}")
print(f"Recalln (Weighted Among Classes): {recall}")

## 4 Results

## 4.1 All Model Metric Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

models = ['Random Forest', 'XGBoost', 'KNN', 'Feed Forward', 'CNN', 'Transformer']
colors = ['blue', 'green', 'purple', 'red','orange','yellow']

bars = axes[0, 0].bar(models, f1_list, color=colors, alpha=0.7)
axes[0, 0].bar(models, f1_list, color=colors, alpha=0.7)
axes[0, 0].set_title("F1 Scores by Model")
axes[0,0].set_ylim(0.6, 1) 
axes[0,0].bar_label(bars, fmt='%.2f')

bars = axes[0,1].bar(models, acc_list)
axes[0, 1].bar(models, acc_list, color=colors, alpha=0.7)
axes[0, 1].set_title("Accuracy by Model")
axes[0, 1].set_ylim(0.6, 1) 
axes[0,1].bar_label(bars, fmt='%.2f')

bars = axes[1,0].bar(models, prec_list)
axes[1, 0].bar(models, prec_list, color=colors, alpha=0.7)
axes[1, 0].set_title("Precision by Model")
axes[1, 0].set_ylim(0.6, 1) 
axes[1,0].bar_label(bars, fmt='%.2f')

bars = axes[1,1].bar(models, recall_list)
axes[1, 1].bar(models, recall_list, color=colors, alpha=0.7)
axes[1, 1].set_title("Recall by Model")
axes[1, 1].set_ylim(0.6, 1) 
axes[1,1].bar_label(bars, fmt='%.2f')


plt.tight_layout()

plt.show()

## 4.2 Runtime Results

In [ ]:
#runtime vs. accuracy
rf_runtime, xgboost_runtime, knn_runtime = 8.8, 25.63, .66
ff_runtime, cnn_runtime, transformer_runtime = 6.38, 7.72, 49.12

runtimes = [rf_runtime, xgboost_runtime, knn_runtime, ff_runtime, cnn_runtime, transformer_runtime]
labels = ['RF','XGBoost', 'KNN', 'FeedForward', 'CNN', 'Transformer']

for i in range(len(runtimes)):
    plt.scatter(runtimes[i], acc_list[i], label=labels[i])

plt.legend()
plt.title("Runtime vs. Accuracy")
plt.xlabel('Runtime in minutes')
plt.ylabel('Accuracy')
plt.grid()
plt.show()



In [ ]:
plt.figure(figsize=(10, 6)) 
bars = plt.bar(models, runtimes, color=colors, alpha=0.7)  
plt.bar_label(bars, fmt='%.2f')
plt.title("Total Training Runtime")
plt.ylabel('Runtime in minutes')
plt.show()

## 4.3 Bayesian Optimization Graphs

Random Forest Optimization

In [ ]:
forest_data = pd.read_csv('RF_bayes_df.csv')
n_estimators = forest_data['param_n_estimators']
mean_test_score = forest_data['mean_test_score']
mean_test_score = -mean_test_score
sorted_indices = n_estimators.argsort()
n_estimators_sorted = n_estimators.iloc[sorted_indices]
mean_test_score_sorted = mean_test_score.iloc[sorted_indices]
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_sorted, mean_test_score_sorted, marker='o', linestyle='-')
plt.title('Bayesian Optimization for Random Forest')
plt.xlabel('n_estimators')
plt.grid()
plt.ylabel('MSE')
plt.show()

KNN Optimization Graph

In [ ]:
knn_pd = pd.read_csv('knn_opti.csv')

n_neighbors = knn_pd['param_n_neighbors']
mean_test_score = knn_pd['mean_test_score']

accuracy = -mean_test_score  

sorted_indices = n_neighbors.argsort()
n_estimators_sorted = n_neighbors.iloc[sorted_indices]
accuracy_sorted = accuracy.iloc[sorted_indices]

plt.figure(figsize=(10, 6))
plt.plot(n_estimators_sorted, accuracy_sorted, marker='o', linestyle='-')
plt.title('n_neighbors vs MSE')
plt.xlabel('n_neighbors')
plt.ylabel('MSE')
plt.grid()
plt.show()
